In [6]:
import tensorflow as tf
import numpy as np
import scipy.misc
import h5py
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from w2_resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

np.random.seed(1)
tf.random.set_seed(2)

In [7]:
# loading the datasets
with h5py.File("./datasets/train_signs.h5", "r") as file:
    train_set_x_orig = np.array(file["train_set_x"][:])
    train_set_y_orig = np.expand_dims(np.array(file["train_set_y"][:]), 0)
    
with h5py.File("./datasets/test_signs.h5", "r") as file:
    test_set_x_orig = np.array(file["test_set_x"][:])
    test_set_y_orig =  np.expand_dims(np.array(file["test_set_y"][:]), 0)
    classes =  np.array(file["list_classes"][:])

In [8]:
def identity_block(X, f, filters, initializer=random_uniform):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = tf.keras.layers.Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = "valid" , kernel_initializer = initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation("relu")(X)
    
    ### START CODE HERE
    ## Second component of main path (≈3 lines)
    ## Set the padding = 'same'
    X = tf.keras.layers.Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = "same", kernel_initializer = initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation("relu")(X)

    ## Third component of main path (≈2 lines)
    ## Set the padding = 'valid'
    X = tf.keras.layers.Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = "valid", kernel_initializer = initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation("relu")(X)
    ### END CODE HERE

    return X

In [9]:
def convolutional_block(X, f, filters, s = 2, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = tf.keras.layers.Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation('relu')(X)

    ### START CODE HERE
    
    ## Second component of main path (≈3 lines)
    X = tf.keras.layers.Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = "same", kernel_initializer = initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation("relu")(X)

    ## Third component of main path (≈2 lines)
    X = tf.keras.layers.Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = "valid", kernel_initializer = initializer(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3)(X)
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s,s), padding = "valid", kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
    
    ### END CODE HERE

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    return X

In [10]:
def ResNet50(input_shape = (64, 64, 3), classes = 6, training=False):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape 
    X_input = tf.keras.layers.Input(input_shape)

    
    # Zero-Padding
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = tf.keras.layers.Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    ### START CODE HERE
    
    # Use the instructions above in order to implement all of the Stages below
    # Make sure you don't miss adding any required parameter
    
    ## Stage 3 (≈4 lines)
    # `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, 3, [128,128,512], 2)
    
    # the 3 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])

    # Stage 4 (≈6 lines)
    # add `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, 3, [256, 256, 1024], 2)
    
    # the 5 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5 (≈3 lines)
    # add `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, 3, [512, 512, 2048], 2)
    
    # the 2 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D()(X)"
    X = tf.keras.layers.AveragePooling2D(pool_size=(2,2))(X)
    
    ### END CODE HERE

    # output layer
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = tf.keras.models.Model(inputs = X_input, outputs = X)

    return model

In [21]:
tf.keras.backend.set_learning_phase(True)
model = ResNet50(input_shape = train_set_x_orig.shape[1:], classes = len(classes))
print(model.summary)

c:\Users\Spino.shop\Desktop\Trading\Apps\.MotherVenv\lib\site-packages\keras\backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


<bound method Model.summary of <keras.engine.functional.Functional object at 0x000001C37432E910>>


In [22]:
np.random.seed(1)
tf.random.set_seed(2)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00015)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [26]:
x_train = train_set_x_orig / 255.
x_test = test_set_x_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(train_set_y_orig, 6).T
Y_test = convert_to_one_hot(test_set_y_orig, 6).T

model.fit(x_train, Y_train, epochs = 13, batch_size = 32)

Epoch 1/13
34/34 [==============================] - 34s 991ms/step - loss: 0.8950 - accuracy: 0.6713
Epoch 2/13
34/34 [==============================] - 41s 1s/step - loss: 0.4751 - accuracy: 0.8417
Epoch 3/13
34/34 [==============================] - 40s 1s/step - loss: 0.3354 - accuracy: 0.8917
Epoch 4/13
34/34 [==============================] - 41s 1s/step - loss: 0.2051 - accuracy: 0.9259
Epoch 5/13
34/34 [==============================] - 41s 1s/step - loss: 0.1966 - accuracy: 0.9333
Epoch 6/13
34/34 [==============================] - 40s 1s/step - loss: 0.1872 - accuracy: 0.9269
Epoch 7/13
34/34 [==============================] - 40s 1s/step - loss: 0.1047 - accuracy: 0.9657
Epoch 8/13
34/34 [==============================] - 40s 1s/step - loss: 0.1095 - accuracy: 0.9630
Epoch 9/13
34/34 [==============================] - 40s 1s/step - loss: 0.0948 - accuracy: 0.9657
Epoch 10/13
34/34 [==============================] - 41s 1s/step - loss: 0.0795 - accuracy: 0.9704
Epoch 11/13
34/3

In [28]:
preds = model.evaluate(x_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 1s 136ms/step - loss: 2.0872 - accuracy: 0.5833
Loss = 2.0872182846069336
Test Accuracy = 0.5833333134651184


In [29]:
model.save_weights("./datasets/model_weights")

In [33]:
model.load_weights("./datasets/model_weights")
model.fit(x_train, Y_train, epochs = 3, batch_size = 32)

Epoch 1/3
34/34 [==============================] - 35s 1s/step - loss: 0.0846 - accuracy: 0.9676
Epoch 2/3
34/34 [==============================] - 36s 1s/step - loss: 0.0888 - accuracy: 0.9685
Epoch 3/3
34/34 [==============================] - 35s 1s/step - loss: 0.0877 - accuracy: 0.9722


In [34]:
preds = model.evaluate(x_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 1s 126ms/step - loss: 1.3882 - accuracy: 0.6750
Loss = 1.3882333040237427
Test Accuracy = 0.675000011920929
